In [1]:
from PIL import Image
from dm_control import composer
import numpy as np
from dm_control import mjcf
from dm_control.composer import variation
from dm_control.composer.observation import observable
from dm_control.composer.variation import distributions, noises
from dm_control.locomotion.arenas import floors

In [2]:

class Leg(object):
    """A 2-DoF leg with position actuators."""

    def __init__(self, length, rgba):
        self.model = mjcf.RootElement()

        # Defaults:
        self.model.default.joint.damping = 2
        self.model.default.joint.type = "hinge"
        self.model.default.geom.type = "capsule"
        self.model.default.geom.rgba = rgba  # Continued below...

        # Thigh:
        self.thigh = self.model.worldbody.add("body")
        self.hip = self.thigh.add("joint", axis=[0, 0, 1])
        self.thigh.add("geom", fromto=[0, 0, 0, length, 0, 0], size=[length / 4])

        # Hip:
        self.shin = self.thigh.add("body", pos=[length, 0, 0])
        self.knee = self.shin.add("joint", axis=[0, 1, 0])
        self.shin.add("geom", fromto=[0, 0, 0, 0, 0, -length], size=[length / 5])

        # Position actuators:
        self.model.actuator.add("position", joint=self.hip, kp=10)
        self.model.actuator.add("position", joint=self.knee, kp=10)


BODY_RADIUS = 0.1
BODY_SIZE = (BODY_RADIUS, BODY_RADIUS, BODY_RADIUS / 2)
random_state = np.random.RandomState(42)


def make_creature(num_legs):
    """Constructs a creature with `num_legs` legs."""
    rgba = random_state.uniform([0, 0, 0, 1], [1, 1, 1, 1])
    model = mjcf.RootElement()
    model.compiler.angle = "radian"  # Use radians.

    # Make the torso geom.
    model.worldbody.add(
        "geom", name="torso", type="ellipsoid", size=BODY_SIZE, rgba=rgba
    )

    # Attach legs to equidistant sites on the circumference.
    for i in range(num_legs):
        theta = 2 * i * np.pi / num_legs
        hip_pos = BODY_RADIUS * np.array([np.cos(theta), np.sin(theta), 0])
        hip_site = model.worldbody.add("site", pos=hip_pos, euler=[0, 0, theta])
        leg = Leg(length=BODY_RADIUS, rgba=rgba)
        hip_site.attach(leg.model)

    return model


In [3]:
class CreatureObservables(composer.Observables):
    @composer.observable
    def joint_positions(self):
        all_joints = self._entity.mjcf_model.find_all("joint")
        return observable.MJCFFeature("qpos", all_joints)

    @composer.observable
    def joint_velocities(self):
        all_joints = self._entity.mjcf_model.find_all("joint")
        return observable.MJCFFeature("qvel", all_joints)


class Creature(composer.Entity):
    """A multi-legged creature derived from `composer.Entity`."""

    def _build(self, num_legs):
        self._model = make_creature(num_legs)

    def _build_observables(self):
        return CreatureObservables(self)

    @property
    def mjcf_model(self):
        return self._model

    @property
    def actuators(self):
        return tuple(self._model.find_all("actuator"))


In [4]:
creature = Creature(num_legs=4)

In [10]:
creature.observables.joint_positions.enabled

False

In [11]:
creature.actuators

(MJCF Element: <position class="/" joint="//unnamed_joint_0" kp="10"/>,
 MJCF Element: <position class="/" joint="//unnamed_joint_1" kp="10"/>,
 MJCF Element: <position class="/" joint="//unnamed_joint_0" kp="10"/>,
 MJCF Element: <position class="/" joint="//unnamed_joint_1" kp="10"/>,
 MJCF Element: <position class="/" joint="//unnamed_joint_0" kp="10"/>,
 MJCF Element: <position class="/" joint="//unnamed_joint_1" kp="10"/>,
 MJCF Element: <position class="/" joint="//unnamed_joint_0" kp="10"/>,
 MJCF Element: <position class="/" joint="//unnamed_joint_1" kp="10"/>)

In [14]:
print(creature.mjcf_model.to_xml_string())

<mujoco model="unnamed_model">
  <compiler angle="radian"/>
  <default>
    <default class="/"/>
    <default class="unnamed_model/">
      <joint type="hinge" damping="2"/>
      <geom type="capsule" rgba="0.37454011884736249 0.95071430640991617 0.73199394181140509 1"/>
    </default>
    <default class="unnamed_model_1/">
      <joint type="hinge" damping="2"/>
      <geom type="capsule" rgba="0.37454011884736249 0.95071430640991617 0.73199394181140509 1"/>
    </default>
    <default class="unnamed_model_2/">
      <joint type="hinge" damping="2"/>
      <geom type="capsule" rgba="0.37454011884736249 0.95071430640991617 0.73199394181140509 1"/>
    </default>
    <default class="unnamed_model_3/">
      <joint type="hinge" damping="2"/>
      <geom type="capsule" rgba="0.37454011884736249 0.95071430640991617 0.73199394181140509 1"/>
    </default>
  </default>
  <worldbody>
    <geom name="torso" class="/" type="ellipsoid" size="0.10000000000000001 0.10000000000000001 0.050000000000

In [15]:
NUM_SUBSTEPS = 25  # The number of physics substeps per control timestep.


class Button(composer.Entity):
    """A button Entity which changes colour when pressed with certain force."""

    def _build(self, target_force_range=(5, 10)):
        self._min_force, self._max_force = target_force_range
        self._mjcf_model = mjcf.RootElement()
        self._geom = self._mjcf_model.worldbody.add("geom", type="cylinder", size=[0.25, 0.02], rgba=[1, 0, 0, 1])
        self._site = self._mjcf_model.worldbody.add("site", type="cylinder", size=self._geom.size * 1.01, rgba=[1, 0, 0, 0])
        self._sensor = self._mjcf_model.sensor.add("touch", site=self._site)
        self._num_activated_steps = 0

    def _build_observables(self):
        return ButtonObservables(self)

    @property
    def mjcf_model(self):
        return self._mjcf_model

    # Update the activation (and colour) if the desired force is applied.
    def _update_activation(self, physics):
        current_force = physics.bind(self.touch_sensor).sensordata[0]
        self._is_activated = current_force >= self._min_force and current_force <= self._max_force
        physics.bind(self._geom).rgba = [0, 1, 0, 1] if self._is_activated else [1, 0, 0, 1]
        self._num_activated_steps += int(self._is_activated)

    def initialize_episode(self, physics, random_state):
        self._reward = 0.0
        self._num_activated_steps = 0
        self._update_activation(physics)

    def after_substep(self, physics, random_state):
        self._update_activation(physics)

    @property
    def touch_sensor(self):
        return self._sensor

    @property
    def num_activated_steps(self):
        return self._num_activated_steps


class ButtonObservables(composer.Observables):
    """A touch sensor which averages contact force over physics substeps."""

    @composer.observable
    def touch_force(self):
        return observable.MJCFFeature(
            "sensordata",
            self._entity.touch_sensor,
            buffer_size=NUM_SUBSTEPS,
            aggregator="mean",
        )


In [16]:
button = Button()
print(button.mjcf_model.to_xml_string())

<mujoco model="unnamed_model">
  <default>
    <default class="/"/>
  </default>
  <worldbody>
    <geom name="//unnamed_geom_0" class="/" type="cylinder" size="0.25 0.02" rgba="1 0 0 1"/>
    <site name="//unnamed_site_0" class="/" type="cylinder" rgba="1 0 0 0" size="0.2525 0.020199999999999999"/>
  </worldbody>
  <sensor>
    <touch name="//unnamed_sensor_0" site="//unnamed_site_0"/>
  </sensor>
</mujoco>


In [36]:
distributions.Uniform(0, 2 * np.pi)()

0.8618068530369953